In [ ]:
import csv

texts = []
labels = []

with open('../../data/clean/reddit_sentiments.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    has_header = csv.Sniffer().has_header(csvfile.read(1024))
    csvfile.seek(0) 
    if has_header:
        next(reader)  # just skip the header
    for row in reader:
        texts.append(row[0])
        labels.append(row[1])

print("texts:", texts[0])
print("labels:", labels[0])

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zerok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zerok\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:

data = {
    'sentence': ['This film is good', 'This film is bad', 'This film is okay', 'This film is great'],
    'sentiment': ['POS', 'NEG', 'NEU', 'POS']
}
df = pd.DataFrame(data)



In [41]:
# calculate prior probability
prior_prob = df['sentiment'].value_counts(normalize=True)

# calculate the probability of each word given the sentiment
word_prob = df.groupby('sentiment').apply(lambda x: x['sentence'].apply(lambda y: [word.lower() for word in word_tokenize(y) if word.lower() not in stop_words]).explode().value_counts(normalize=True))

word_prob

sentiment  sentence
NEG        film        0.50
           bad         0.50
NEU        film        0.50
           okay        0.50
POS        film        0.50
           good        0.25
           great       0.25
Name: proportion, dtype: float64

In [42]:


def predict_sentiment(sentence):
    words = [word.lower() for word in word_tokenize(sentence) if word.lower() not in stop_words]
    posterior_prob = prior_prob.copy()
    
    for word in words:
        for sentiment, prob in word_prob.items():
            if word == sentiment[1]:
                posterior_prob[sentiment[0]] *= prob
    
    posterior_prob /= posterior_prob.sum()
    
    return posterior_prob.idxmax()

new_sentence = "This film has excellent quality"
predicted_sentiment = predict_sentiment(new_sentence)
print(f"Predicted Emotion: {predicted_sentiment}")


Predicted Emotion: POS
